In [ ]:
from getpass import getpass

CLIENT_ID = getpass("CLIENT_ID_POSTDAM ")
CLIENT_SECRET = getpass("CLIENT_SECRET_POSTDAM: ")
USERNAME = getpass("USERNAME_POSTDAM: ")

In [ ]:
import sys
import os

sys.path.append(os.path.join(os.getcwd(), '..'))

from datenraum import *
from serlo_api_client import fetch_entity_content

In [ ]:
session = Session(Environment.POSTDAM, Credentials(CLIENT_ID, CLIENT_SECRET))
client = Client(session)
source = client.create_source(slug="serlo", name="Serlo Education e.V.", organization="Serlo Education e.V.")

In [ ]:
def get_nodes():
    nodes = []

    while True:
        new_nodes = source.get_nodes(offset=len(nodes))

        nodes.extend(new_nodes)

        if len(new_nodes) == 0:
            break
    
    return nodes

nodes = get_nodes()
print(f"{len(nodes)} nodes downloaded from Postdam Datenraum of Serlo")

In [ ]:
contents = []

for node in nodes:
    serlo_uuid = int(node["id"].replace("https://serlo.org/", ""))
    print(f"fetching content of {serlo_uuid}")
    response = fetch_entity_content(serlo_uuid)
    if "currentRevision" in response["uuid"] and (
        "content" in response["uuid"]["currentRevision"]
    ):
        contents += { 
            "id": node['id']
            "content": response["uuid"]["currentRevision"]["content"] 
        }
    # Let's not DDOS our own api
    time.sleep(0.2)

In [ ]:
with open('public/uuids-content', "w", encoding="utf-8") as output_file:
    json.dump(nodes, output_file)